In [80]:
#hide
#skip
! [ -e /content ] && pip install -Uqq fastai  # upgrade fastai on colab

In [81]:
#export
from fastai.basics import *
from fastai.text.learner import *
from xcube.text.models.core import *

In [82]:
#hide
from nbdev.showdoc import *

In [83]:
#default_exp text.learner

# Learner for the text application

## Helper functions

In [84]:
#export
from xcube.text.models.core import _model_meta 

In [85]:
#export
def _get_text_vocab(dls):
    vocab = dls.vocab
    if isinstance(vocab, L): vocab = vocab[0]
    return vocab

## `Learner` convenience functions

In [86]:
#export 
@delegates(Learner.__init__)
def text_classifier_learner(dls, arch, seq_len=72, config=None, backwards=False, pretrained=True, drop_mult=0.5, n_out=None,
                           lin_ftrs=None, ps=None, max_len=72*20, y_range=None, **kwargs):
    "Create a `Learner` with a text classifier from `dls` and `arch`."
    vocab = _get_text_vocab(dls)
    if n_out is None: n_out = get_c(dls)
    assert n_out, "`n_out` is not defined, and could not be inferred from the data, set `dls.c` or pass `n_out`"
    model = get_text_classifier(arch, len(vocab), n_out, seq_len=seq_len, config=config, y_range=y_range,
                                drop_mult=drop_mult, lin_ftrs=lin_ftrs, ps=ps, max_len=max_len)
    meta = _model_meta[arch]
    learn = TextLearner(dls, model, splitter=meta['split_clas'], **kwargs)
    url = 'url_bwd' if backwards else 'url'
    if pretrained:
        if url not in meta:
            warn("There are no pretrained weights for that architecture yet!")
            return learn
        model_path = untar_data(meta[url], c_key='model')
        try: fnames = [list(model_path.glob(f'*.{ext}'))[0] for ext in ['pth', 'pkl']]
        except IndexError: print(f'The model in {model_path} is incomplete, download again'); raise
        learn = learn.load_pretrained(*fnames, model=learn.model[0])
        learn.freeze()
    return learn   

In [87]:
from nbdev.export import notebook2script; notebook2script()

Converted 00_core.ipynb.
Converted 01_layers.ipynb.
Converted 02_text.models.core.ipynb.
Converted 03_text.learner.ipynb.
Converted 04_metrics.ipynb.
Converted index.ipynb.


In [88]:
CAML3??

Init signature: CAML3(dims, ps, bptt, y_range=None)
Docstring:      Same as `nn.Module`, but no need for subclasses to call `super().__init__`
Source:        
class CAML3(Module):
    def __init__(self, dims, ps, bptt, y_range=None):
        self.lbs = dims[-1]
        self.fts = dims[0]//3
        # self.layers = LinBnDrop(self.lbs, self.fts, p=ps, act=None) # wrong_deb
        self.layers = LinBnDrop(self.lbs, ln=False, p=ps, act=None) # deb
        self.bptt = bptt
        self.emb_label = nn.Embedding(self.lbs, self.fts) # deb
        self.final_lin = nn.Linear(self.fts, self.lbs)

    def forward(self, input):
        out, _ = input
        # breakpoint()
        # x = masked_concat_pool(out, mask, self.bptt)
        attn_wgts = out @ self.emb_label.weight.transpose(0, 1) # deb
        attn_wgts = F.softmax(attn_wgts, 1) # deb
        # attn_wgts = torch.sigmoid(attn_wgts) # deb
        ctx = attn_wgts.transpose(1,2) @ out # deb

        x = self.layers(ctx)
        x = self.final